HTML is the basic language used to create a web page.
HTML is very structured/hierarchical.
Every page is made up of discrete "elements".
Elements are labeled with "tags"

***
<html>
    <head>
    </head>

    <body>
        <p class="red"> Some text. You are beginning to learn HTML. </p>
        <h1> This is a header </h1>
        <a href="www.google.com"> Some link </a>
        

    </body>

</html>


### Get the HTML from a page and convert to a BeautifulSoup object


In [35]:
import urllib2

from bs4 import BeautifulSoup

url = "http://www.boxofficemojo.com/movies/?id=biglebowski.htm"
page = urllib2.urlopen(url)
soup = BeautifulSoup(page)

In [3]:
# print soup

## soup.find()
Search for a type of tag by using the tag as a string (like 'body', 'div', 'p', 'a', etc) as an argument)

In [4]:
print soup.find("a")

<a href="/goto.php?a=5" target="4"><font face="Verdana" size="3"><b>'Furious 7' hits $800 million worldwide... &gt;</b></font><br/></a>


In [5]:
# print soup.a
print soup.a.prettify()

<a href="/goto.php?a=5" target="4">
 <font face="Verdana" size="3">
  <b>
   'Furious 7' hits $800 million worldwide... &gt;
  </b>
 </font>
 <br/>
</a>



In [6]:
# soup.find_all() returns a list of all matches.
# for link in soup.find_all('a'):
#     print link

# retrive the url from an anchor tag
print soup.find('a')
print
print soup.find('a')['href']

<a href="/goto.php?a=5" target="4"><font face="Verdana" size="3"><b>'Furious 7' hits $800 million worldwide... &gt;</b></font><br/></a>

/goto.php?a=5


In [8]:
# you can match on an attribute like a class, or an id
# for element in soup.find_all()

soup.find(class_="mp_box_content").find_all('td')

[<td width="40%"><b>Domestic:</b></td>,
 <td align="right" width="35%"> <b>$17,451,873</b></td>]

In [9]:
# print soup.find(id="hp_footer")

## Items to scrape for in each movie:

### movie title, total domestc gross, release date, runtime, rating

In [27]:
# Movie title
print soup.find("title")
title_string = soup.find("title").text
print title_string.split("(")[0].strip()
title_string = title_string.split("(")[0].strip()

<title>The Big Lebowski (1998) - Box Office Mojo</title>
The Big Lebowski


In [19]:
# Domestic Total Gross (exact search: include white spaces)
print soup.find(text="Domestic Total Gross:")

# using regular expressions you don't have to be exact (no white spaces)
import re
dtg_string = soup.find(text=re.compile("Domestic Total Gross"))
print dtg_string.findNextSibling().text

None
$17,451,873


In [20]:
def get_movie_value(soup, field_name):
    """
    takes a string attribute of a movie on the page, and
    returns the string in the next sibling object (the value for that attribute)
    """
    obj = soup.find(text = re.compile(field_name))
    if not obj:
        return None
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text
    else:
        return None

In [28]:
print title_string

dtg = get_movie_value(soup, "Domestic Total")
print dtg

# run time
runtime = get_movie_value(soup, "Runtime")
print runtime

# rating
rating = get_movie_value(soup, "MPAA Rating")
print rating

# release data
release_date = get_movie_value(soup, "Release Date")
print release_date

The Big Lebowski
$17,451,873
1 hrs. 57 min.
R
March 6, 1998


In [29]:
headers = ["movie title", "domestic total gross", "release date", "runtime", "rating"]
movie_data = []

movie_dict = dict(zip(headers, [title_string, dtg, release_date, runtime, rating]))
# print movie_dict

movie_data.append(movie_dict)
print movie_data


[{'rating': u'R', 'runtime': u'1 hrs. 57 min.', 'movie title': u'The Big Lebowski', 'domestic total gross': u'$17,451,873', 'release date': u'March 6, 1998'}]


## Searching movies starting with "B...", first page list

In [131]:
import urllib2
import re
from bs4 import BeautifulSoup

url = "http://www.boxofficemojo.com/movies/alphabetical.htm?letter=B&p=.htm"
page = urllib2.urlopen(url)
soup = BeautifulSoup(page)

In [79]:
# print soup

In [132]:
movielinks = []

if soup.find_all('a', href=re.compile('movies/\?id=b.*.htm')):
    for link in soup.find_all('a', href=re.compile('movies/\?id=b.*.htm')):
        movielink = "http://www.boxofficemojo.com" + str(link['href'])
        movielinks.append(movielink)
        
print len(movielinks)
# print movielinks

TotalGross = 0

for link in movielinks:
    url = link
    page = urllib2.urlopen(url)
    soup = BeautifulSoup(page)
    # print(soup)
    dtg_string = soup.find(text=re.compile("Domestic Total Gross"))
    # print dtg_string
    if dtg_string:
        GrossString = dtg_string.findNextSibling().text
        print GrossString
        TotalGross += int(GrossString.replace(',', '')[1:])
        print TotalGross

print TotalGross

214
$39,371
39371
$7,246,735
7286106
$365,727
7651833
$476,270
8128103
$660,216
8788319
$89,672
8877991
$950,792
9828783
$1,305,187
11133970
$17,530,973
28664943
$63,658,910
92323853
$18,319,860
110643713
$34,302,837
144946550
$4,398,938
149345488
$7,320,323
156665811
$17,234
156683045
$26,712,476
183395521
$28,734,552
212130073
$27,250,736
239380809
$60,494,212
299875021
$16,827,402
316702423
$1,867,792
318570215
$9,685,976
328256191
$14,972,297
343228488
$22,532,572
365761060
$7,889
365768949
$44,852
365813801
$21,760,240
387574041
$38,435,947
426009988
$447,954
426457942
$702,166
427160108
$12,139
427172247
$569,280
427741527
$11,809,387
439550914
$312,954
439863868
$503
439864371
$91,258,000
531122371
$13,110,903
544233274
$210,609,762
754843036
$118,450,002
873293038
$87,727,583
961020621
$37,490,007
998510628
$2,392,589
1000903217
$77,868,585
1078771802
$12,343
1078784145
$1,191,835
1079975980
$12,663
1079988643
$153,068
1080141711
$3,200
1080144911
$5,812
1080150723
$3,731
10801